In [1]:
! pip install ..
import numpy as np
import math
from scipy.stats import poisson
from dismal import matrices, likelihood2
from scipy import linalg
from dismal.likelihood2 import *

Processing /Users/s2341012/Dropbox/DISMaL_chapter/DISMaL
  Preparing metadata (setup.py) ... done
  Created wheel for dismal: filename=dismal-0.1-py3-none-any.whl size=15346 sha256=d15371606a7f56ddeb5732c289ad1b593364cf45657f186a84cbee48cd499104
  Stored in directory: /private/var/folders/qk/10crn8pd5_d038f0tw_7z2800000gp/T/pip-ephem-wheel-cache-nfhnma9k/wheels/10/c9/59/9e87bfb89308645a07e4be9051b0645d49425b51da0dd9ddc7
Successfully built dismal
  Attempting uninstall: dismal
    Found existing installation: dismal 0.1
    Uninstalling dismal-0.1:
      Successfully uninstalled dismal-0.1


In [7]:
len([i for i in [False, False] if i])

0

In [2]:
q1 = matrices.GeneratorMatrix(c1=1, c2=1, m1_prime=0, m2_prime=0, matrix_type="Q1")
q2 = matrices.GeneratorMatrix(b=1, m1=0, m2=0, matrix_type="Q2")
q3 = matrices.GeneratorMatrix(a=1, matrix_type="Q3")

In [3]:
S = [[1] * 10] * 3

In [4]:
composite_neg_ll([1,1,1,1,1,1,1,0,0,0,0], S)

45.49445170925571

In [3]:
S = [[0], [0], [0]]
i = 0
t1 = 1
v = 1
g, eigvals_q1 = q1.eigen()
c, eigvals_q2 = q2.eigen()

# reorder for CWH
g_prime = np.array([g[0], g[1], g[3], g[2]])
c_prime = np.array([c[0], c[1], c[3], c[2]])

ginv = linalg.inv(g_prime)
cinv = linalg.inv(c_prime)
alpha = -eigvals_q1[0:3]
beta = -eigvals_q2[0:3]
gamma = 1/q3.pop_size1 #1/a

gg = -ginv @ np.diag(g_prime[:,3])
cc = -cinv @ np.diag(c_prime[:,3])
pij1 = p_matrix(g_prime, ginv, eigvals_q1, t1)
pij2 = p_matrix(c_prime, cinv, eigvals_q2, v)


In [5]:
neg_likelihood(q1, q2, q3, 1, 1, s_vals=[0], state=1)

0.6931471805599453

In [27]:
neg_likelihood(q1, q2, q3, 1, 1, s_vals=[0], state=1)

0.6931471805599453

In [30]:
lls = []
for i in [1,2,3]:
    lls.append([neg_likelihood(q1, q2, q3, 1, 1, s_vals=[s], state=i) for s in range(0,10)])

In [31]:
lls

[[0.6931471805599453,
  1.3862943611198906,
  2.0794415416798357,
  2.772588722239781,
  3.4657359027997265,
  4.1588830833596715,
  4.852030263919617,
  5.545177444479562,
  6.238324625039508,
  6.931471805599453],
 [0.6931471805599453,
  1.3862943611198906,
  2.0794415416798357,
  2.772588722239781,
  3.4657359027997265,
  4.1588830833596715,
  4.852030263919617,
  5.545177444479562,
  6.238324625039508,
  6.931471805599453],
 [2.6931471805599454,
  1.7768564486857903,
  1.514492184218299,
  1.6085211338665755,
  1.9296192032382007,
  2.4007885602242625,
  2.969321646160246,
  3.5976647311742607,
  4.259919560738974,
  4.939637295454291]]

In [33]:
[-np.log(1/(2**n)) for n in range(0,10)]

[-0.0,
 0.6931471805599453,
 1.3862943611198906,
 2.0794415416798357,
 2.772588722239781,
 3.4657359027997265,
 4.1588830833596715,
 4.852030263919617,
 5.545177444479562,
 6.238324625039508]